# InfCtx trainer validation
This model being trained has the same settings as raven 1B5 model.
- Layer count: 24
- Embed size: 2048

The goal is to validate loss rate change, across the exact same hyper parameters with the following
- 1024 data chunk size
- same learningrate / weightdecay / seed
- "teven/enwiki_10k" dataset, chunked to 1024 token sizes

With only the change in training context size
- 1024 context vs 128 context

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps
>
> And that you have completed the `baseline-setup.ipynb`

## Configure and apply your preferred settings

Adjust your desired deepspeed settings, and gpu device count.

Enable/Disable WANDB here as well ( Enabled by default, as we need the loss curve for this experiment )

( note you will need to rerun this cell, if you restart your env )

In [8]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
WANDB_ENABLE=True
WANDB_PREFIX="bptt-test"

print("WANDB_PREFIX:", WANDB_PREFIX)
print("WANDB_ENABLE:", WANDB_ENABLE)
print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("GPU_DEVICES:", GPU_DEVICES)

if WANDB_ENABLE:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

WANDB_PREFIX: bptt-test
WANDB_ENABLE: True
DEEPSPEED_STRAT: deepspeed_stage_2_offload
GPU_DEVICES: auto


# (optional) Baseline full context (1024) training

(you can skip this, and use the optional baseline found in `baseline-setup.ipynb`)

Perform a full 1 epoch training run of training context size = 1024. Ensuring all data samples fit within the allocated training size.

In [12]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 new_train.py fit \
        -c ../notebook/trainer-validation/config/baseline-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (full, train-ctx=1024, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-compile' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230711_110127-wdl50h88
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run bptt-test (full, train-ctx=1024, data-ctx=1024, deepspeed_stage_2_offload)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/wdl50h88
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_1024_bf16/build.ni

# Back-Propagation Through Time (512) training

Perform a full 1 epoch training run of training context size = 512. This is a less exegerated version of the 128 training

In [ ]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 new_train.py fit \
        -c ../notebook/trainer-validation/config/baseline-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (bptt, train-ctx=512, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.ctx_len=512

[2023-07-10 09:40:47,658] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-native' with torch '2.1.0.dev20230706'
Global seed set to 3941088705
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_512_bf16/build.ninja...
Building extension module wkv_512_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_512_bf16...
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/connector.py:555: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using

# Back-Propagation Through Time (128) training

Perform a full 1 epoch training run of training context size = 128. Forcing all data samples to be segmented 8 times, via "Truncated Back-Propagation Through Time"

In [ ]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 new_train.py fit \
        -c ../notebook/trainer-validation/config/baseline-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (bptt, train-ctx=128, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.ctx_len=128

[2023-07-10 09:47:36,054] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-native' with torch '2.1.0.dev20230706'
Global seed set to 3941088705
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_128_bf16...
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/fabric/connector.py:555: UserWarning: bf16 is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using

# Last segmented (128) training

Finally perform a full 1 epoch training run of training context size = 128. Only using the last segment. (This replicates previous known regression)

In [ ]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 new_train.py fit \
        -c ../notebook/trainer-validation/config/baseline-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (last-bptt, train-ctx=128, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.bptt_learning_range=1 \
        --model.ctx_len=128

[2023-07-01 22:13:03,001] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230701_221304-w0k4pf4n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-validation-last-segment (train-ctx=128, data-ctx=1024, bs=12)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/w0k4pf4n
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_128_bf16/build.ninja...
Building extension module wkv_128_bf16...
Allowing ninja to set a default number of workers... (overridab